# Tutorial 02: Secure Setup

Run OpenClaw safely in Docker with proper security hardening.

## Overview 🔎

In this tutorial, you'll:
1. Understand why Docker isolation is critical
2. Set up OpenClaw using the official Docker configuration
3. Configure security hardening
4. Verify everything is working

**Time**: ~10 minutes (longer on first build)

## Why This Matters

From the [OpenClaw documentation](https://docs.openclaw.ai/install/docker):

> ⚠️ **WARNING**: Never run OpenClaw with root or Administrator privileges directly on your host machine. An AI agent can make mistakes. A simple hallucinated command like `rm -rf /` could be catastrophic.

The risks are real:

| Risk | Impact |
|------|--------|
| **Prompt Injection** | 2.6% of Moltbook posts contain attacks |
| **Remote Code Execution** | Malicious prompts can execute shell commands |
| **Data Exfiltration** | Agent has access to your filesystem |
| **Persistent Attacks** | Memory enables delayed exploitation |

Docker provides isolation: even if the agent goes rogue, it can only damage the container.

## Prerequisites

### 1. Docker Desktop or Docker Engine

Check if Docker is installed:

In [ ]:
!docker --version

If not installed:
- **Mac**: [Docker Desktop for Mac](https://docs.docker.com/desktop/install/mac-install/)
- **Windows**: [Docker Desktop for Windows](https://docs.docker.com/desktop/install/windows-install/) + WSL2
- **Linux**: [Docker Engine](https://docs.docker.com/engine/install/)

### 2. Docker Compose V2

In [ ]:
!docker compose version

### 3. Check Docker is Running

In [ ]:
!docker info > /dev/null 2>&1 && echo "✓ Docker is running" || echo "✗ Start Docker Desktop first"

## Implementation 🛠️

### Option A: Use Our Script (Recommended)

From the repo root, run:

```bash
./start.sh
```

This handles everything automatically.

### Option B: Manual Setup

If you prefer to understand each step:

#### Step 1: Clone OpenClaw

```bash
git clone https://github.com/openclaw/openclaw.git ~/.openclaw-docker
cd ~/.openclaw-docker
```

#### Step 2: Build the Docker Image

```bash
docker build -t openclaw:local -f Dockerfile .
```

This takes a few minutes on first run. The image includes:
- Node.js 22 runtime
- OpenClaw gateway and CLI
- Non-root user (UID 1000)

#### Step 3: Run Onboarding

```bash
docker compose run --rm openclaw-cli onboard
```

The wizard configures:
- **Gateway mode**: Local (recommended)
- **AI Provider**: Choose Anthropic (Claude) or OpenAI
- **Authentication**: API key or OAuth
- **Daemon**: Install as background service

**Tip**: For OpenAI Codex OAuth, you'll see a localhost error in the browser. Copy that full URL back into the terminal.

#### Step 4: Start the Gateway

```bash
docker compose up -d openclaw-gateway
```

The `-d` flag runs it in the background.

#### Step 5: Verify It's Running

In [ ]:
!docker ps --filter "name=openclaw" --format "table {{.Names}}\t{{.Status}}\t{{.Ports}}"

Expected output:
```
NAMES                              STATUS          PORTS
openclaw-docker-openclaw-gateway-1 Up 2 minutes    0.0.0.0:18789->18789/tcp
```

## Security Hardening

The default `docker-compose.yml` provides basic isolation. For production use, add these flags:

### Recommended Security Configuration

```yaml
services:
  openclaw-gateway:
    # ... existing config ...
    security_opt:
      - no-new-privileges:true
    cap_drop:
      - ALL
    cap_add:
      - CHOWN
      - DAC_OVERRIDE
      - NET_BIND_SERVICE
    read_only: true
    tmpfs:
      - /tmp
    ports:
      - "127.0.0.1:18789:18789"  # Localhost only!
```

### What These Flags Do

| Flag | Protection |
|------|------------|
| `no-new-privileges` | Prevents privilege escalation attacks |
| `cap_drop: ALL` | Removes all Linux capabilities |
| `cap_add: ...` | Only adds minimal required capabilities |
| `read_only: true` | Container can't modify its own filesystem |
| `127.0.0.1:18789` | Dashboard only accessible from localhost |

## Agent Sandboxing (Advanced)

OpenClaw supports running non-main sessions in isolated Docker containers. This is critical for Moltbook because your agent processes untrusted content.

### Enable Sandbox Mode

Edit `~/.openclaw/config.json`:

```json
{
  "agents": {
    "defaults": {
      "sandbox": {
        "mode": "non-main",
        "docker": {
          "image": "openclaw-sandbox:bookworm-slim",
          "network": "none",
          "memory": "1g",
          "cpus": 1,
          "readOnlyRoot": true,
          "capDrop": ["ALL"]
        }
      }
    }
  }
}
```

### Build Sandbox Image

```bash
cd ~/.openclaw-docker
scripts/sandbox-setup.sh
```

## Access the Dashboard

Open your browser to: **http://127.0.0.1:18789/**

### Get Your Token

The dashboard requires authentication:

In [ ]:
# Run this from ~/.openclaw-docker directory
!cd ~/.openclaw-docker && docker compose run --rm openclaw-cli dashboard --no-open 2>/dev/null || echo "Run from terminal: cd ~/.openclaw-docker && docker compose run --rm openclaw-cli dashboard --no-open"

Copy the token and paste it into the dashboard login.

## Managing Your Installation

### View Logs

```bash
cd ~/.openclaw-docker
docker compose logs -f openclaw-gateway
```

Or use our script: `./logs.sh`

### Stop the Agent

```bash
docker compose down
```

Or: `./stop.sh`

### Restart

```bash
docker compose restart openclaw-gateway
```

Or: `./restart.sh`

### Health Check

```bash
docker compose run --rm openclaw-cli doctor
```

### Completely Remove

```bash
docker compose down -v  # Removes volumes too
rm -rf ~/.openclaw      # Removes all config
rm -rf ~/.openclaw-docker
```

## Volume Mounts

Understanding what's persisted:

| Path | Purpose |
|------|--------|
| `~/.openclaw/` | Config, credentials, memory, skills |
| `~/openclaw/workspace/` | Agent's working directory |
| `~/.openclaw/skills/` | Installed skills (including Moltbook) |

These directories survive container restarts and rebuilds.

## Troubleshooting

### "Permission denied" errors

The container runs as UID 1000. Fix ownership:

```bash
sudo chown -R 1000:1000 ~/.openclaw
```

### OAuth localhost error

For headless servers or WSL2, the OAuth redirect shows a browser error. Copy the **full URL** (including the error page URL) and paste it back into the terminal.

### "Port 18789 already in use"

```bash
# Find what's using it
lsof -i :18789

# Or change the port in docker-compose.yml
```

### Container keeps restarting

Check logs for errors:

```bash
docker compose logs openclaw-gateway
```

Common issues:
- Invalid API key
- Missing configuration files
- Network connectivity

## Verify Installation

In [ ]:
import requests

try:
    response = requests.get("http://127.0.0.1:18789/health", timeout=5)
    if response.status_code == 200:
        print("✓ OpenClaw gateway is running!")
        print(f"  Dashboard: http://127.0.0.1:18789/")
    else:
        print(f"Gateway returned status {response.status_code}")
except requests.exceptions.ConnectionError:
    print("✗ Gateway not responding. Run ./start.sh first.")
except Exception as e:
    print(f"Error: {e}")

## What's Next

Your secure environment is ready. Now let's create your Moltbook agent:

**[Tutorial 03: Your First Agent](03_first_agent.ipynb)** - Register on Moltbook and make your first post.